# Multirotor trajectory optimization with openmdao and FMU

In [1]:
import os.path as pth
import numpy as np
import matplotlib.pyplot as plt
import openmdao.api as om

Unable to import mpi4py. Parallel processing unavailable.
Unable to import petsc4py. Parallel processing unavailable.
Unable to import petsc4py. Parallel processing unavailable.


In [2]:
# Loading the different FMUs

from pyfmi import load_fmu

ressources_folder = "../ressources"
speed_model_file_name = "DroneFMU.DroneOptim_Speed.fmu"
torque_model_file_name = "DroneFMU.DroneOptim_Torque.fmu"

speed_file_path = pth.join(ressources_folder, speed_model_file_name)
torque_file_path = pth.join(ressources_folder, torque_model_file_name)

#Load the FMU
speed_model = load_fmu(speed_file_path) # Model
torque_model = load_fmu(torque_file_path) # Model

## Running the problem

In [3]:
from trajectory import Model

num_points = 100
num_simu_points = 500
final_time = 5.
use_torque = True
model = torque_model

# Initial torque
torque = np.linspace(1., 1., num=num_points)

prob = om.Problem()
prob.model = Model(
                num_points=num_points,
                num_simu_points=num_simu_points,
                final_time=final_time,
                use_torque=use_torque,
                model=model,
)

prob.setup(check=True)

prob["torque"] = torque

prob.run_model()


INFO: checking out_of_order
INFO: checking system
INFO: checking solvers
INFO: checking dup_inputs
INFO: checking missing_recorders
INFO: checking comp_has_no_outputs
INFO: checking auto_ivc_warnings
Simulation interval    : 0.0 - 5.0 seconds.
Elapsed simulation time: 0.03487829999999992 seconds.


In [4]:
prob['position']

array([ 0.00000000e+00, -1.39962586e-18, -6.28445867e-18, -1.46544984e-17,
       -2.65097452e-17, -4.18501989e-17, -6.06758595e-17, -8.29867271e-17,
       -1.08782802e-16, -1.38064083e-16, -1.70830572e-16, -2.07082267e-16,
       -2.46819169e-16, -2.90041279e-16, -3.36748595e-16, -3.86941118e-16,
       -4.40618849e-16, -4.97781786e-16, -5.58429930e-16, -6.22563281e-16,
       -6.90181839e-16, -7.61285604e-16, -8.35874576e-16, -9.13948755e-16,
       -9.95508141e-16, -1.08055412e-15, -1.16908946e-15, -1.26111001e-15,
       -1.35661577e-15, -1.45560673e-15, -1.55808290e-15, -1.66404428e-15,
       -1.77349087e-15, -1.88642266e-15, -2.00283966e-15, -2.12274186e-15,
       -2.24612927e-15, -2.37300189e-15, -2.50335972e-15, -2.63720275e-15,
       -2.77453099e-15, -2.91534444e-15, -3.05964309e-15, -3.20742696e-15,
       -3.35869602e-15, -3.51345030e-15, -3.67168978e-15, -3.83341447e-15,
       -3.99862437e-15, -4.16731947e-15, -4.33950255e-15, -4.51517361e-15,
       -4.69432988e-15, -